# Basic Data Exploration

In [ ]:
import arpes
from arpes.io import example_data

f = example_data.cut

In [ ]:
f.S.plot()

Although we can plot the spectrum off of a `Dataset`, which might contain additional attributes like the photocurrent, beam current, temperature, or wall-clock time, by using `.S.plot()`, we can also get the spectrum attribute from the full data:

In [ ]:
f.spectrum

## Selecting Data

Typically, we will not want to use the full spectrometer window. Instead we will want to focus on a particular energy or momentum range, such as the region just above, below, or around the chemical potential, or at the energy distribution curves (EDCs) around a particular significant momentum. Data selections can be performed either on `Dataset`s or `DataArray`s. In the former case, it has the effect of selecting on a particular dimension or set of dimensions for each attribute that has those dimensions.

In general, we do not need to know the indices of the region we want to select: we can use the physically meaningful coordinates. As an example, to get the band structure from our data `f` between the ground reference of the spectrometer (the chemical potential) and a point 200 millivolts below, we can use `.sel`.

In [ ]:
f.sel(eV=slice(-0.2, 0)).S.plot()

`.sel` accepts any number of dimensions specified by a `slice`. The arguments to the slice provide a low and high cutoff respectively.

We can also select just a single point along an axis by passing a value instead of a slice. In general we will want to be safe and pass the `method='nearest'` argument, in order to ensure that if the exact value we requested is not included, we will get the nearest pixel.

In [ ]:
# get the EDC nearly at the gamma point
f.sel(phi=0.4, method="nearest").S.plot()

Of course, we can select over a wider window for better statistics, so long as we average or sum out the angle axis after selecting.

In [ ]:
# get the EDC nearly at the gamma point
f.sel(phi=slice(0.38, 0.42)).sum("phi").S.plot()

### Selecting with array indices

In instances where you would like to subselect by an index (ranging above from 0 to 239), as opposed to the physically significant coordinate value, we can use `.isel` instead.

In [ ]:
f.isel(phi=40).S.plot()

### Open ended selections

Selections can be made open ended on one or both sides by passing `None`. In the follow example, the inclusion of `None` will cause the selection to continue to the end of the data on the high end, i.e pixels 40 to 240 on the 'phi' axis.

In [ ]:
f.isel(phi=slice(40, None)).S.plot()

## Summing and Averaging

We saw above an example of taking a binned EDC by selecting a narrow angular region and taking the sum over that axis. We can also request means or sums over multiple axes with `.mean` and `.sum`.

In [ ]:
f.mean("eV", keep_attrs=True).S.plot()

In [ ]:
# to get the value as a plain float instead of a scalar dataset
# use `.mean(["eV", "phi"]).item()` instead
f.spectrum.mean(["eV", "phi"], keep_attrs=True)

**Caveat**: Summing an axis will remove all attributes unless the `keep_attrs=True` parameter is included in the call to `.sum`. Forgetting to do this where it is necessary can cause issues down the line for analysis that requires access to the attributes, such as in converting to momentum space, where the photon energy and coordinate offset for normal emission are stored as attributes.

## Transposing

To transpose axes, you can use `.transpose` or `.T`. In `xarray` and therefore in PyARPES, you interact with data using named dimensions. As a result, transposing is rarely necessary except to set axis order when plotting.

In [ ]:
f.transpose("eV", "phi").S.plot()  # equivalently, f.T.S.plot()

## Interactive Data Browsing

Facilities are included for doing interactive data exploration and analysis both directly in Jupyter, by using holoview via Bokeh. To learn more about interactive analysis, please read the interactive analysis section of the documentation.

In [ ]:
f.spectrum.S.show()

## Learning more about data manipulation

PyARPES uses the excellent [xarray](https://xarray.pydata.org) to provide a rich "Wave"-like data model. For non-spectroscopy related
analysis tasks, you should read their documentation.

### PyARPES philosophy on interactive tools

Instead of one large interactive application where you perform analysis, PyARPES has many small interactive utilities which are built for a single purpose. From Ver 4.0.1 (pragmatically, from 4.0), the Qt based interactive tools are depricated. Thus, the following tools cannot be used:

1. A data browser: `arpes.plotting.qt_tool`
2. A momentum offset browser: `arpes.plotting.qt_ktool.ktool`
3. A tool for path selection: `arpes.plotting.basic_tools.path_tool`
4. A tool for mask selection: `arpes.plotting.basic_tools.mask_tool`
5. A tool to show a slice on a BZ recorded by the spectrometer or scan: `arpes.plotting.bz_tool.bz_tool`

and others.

Instead, holoviews based tools are prepared as necessary.

1. For checking the intensity at the specific position: `arpes.plotting.profile_view`


PyARPES prefers simpler specific tools because it makes them easier to write and maintain, and because PyARPES prefers you do as much analysis via the notebook as possible. This makes for better, more reproducible science. That's why I have dropped Qt-based UI tools.

## Exercises

### Learning more about `xarray`

1. Skim the [data structures guide](https://xarray.pydata.org/en/stable/user-guide/data-structures.html) at the `xarray` docs.
2. Open a PyARPES Jupyter cell and print one of the piece of example data with `from arpes.io import example_data; example_data.cut` or similar. Make sure you can understand the correspondence between the [data structures guide](https://xarray.pydata.org/en/stable/user-guide/data-structures.html) and ARPES idioms.


### Practicing data selection

1. What happens if you select a range backward?
2. What happens to attributes and coordinates when you select a single value or range?

### Practicing interactive browsing
1. Try using `profile_view` to find the projection of the gamma point in `example_data.map`? 

